## makemore: babynames lotto tflite

In [1]:
from time import time
myseed=round(time())

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
words = open('sat_lotto.txt', 'r').read().splitlines()
words[:8]

['37,28,27,11,41,23,2,1',
 '11,9,14,29,7,15,17,23',
 '9,37,32,41,40,16,28,30',
 '6,10,18,32,33,37,21,23',
 '14,16,24,34,41,44,21,30',
 '5,10,30,32,34,42,19,45',
 '1,2,18,24,36,43,9,31',
 '3,14,32,37,42,45,17,20']

In [4]:
len(words)

820

In [5]:
results = []

for w in words:
    results.append([int(num) for num in w.strip().split(',')])

In [6]:
results[:8]

[[37, 28, 27, 11, 41, 23, 2, 1],
 [11, 9, 14, 29, 7, 15, 17, 23],
 [9, 37, 32, 41, 40, 16, 28, 30],
 [6, 10, 18, 32, 33, 37, 21, 23],
 [14, 16, 24, 34, 41, 44, 21, 30],
 [5, 10, 30, 32, 34, 42, 19, 45],
 [1, 2, 18, 24, 36, 43, 9, 31],
 [3, 14, 32, 37, 42, 45, 17, 20]]

In [7]:
chars = sorted(list(set(range(0,46))))
stoi = {s:i for i,s in enumerate(chars)}
# stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(stoi)
print(vocab_size)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45}
46


In [8]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

#abigail a b i, g

def build_dataset(words):  
    block_size = 3
    X,Y = [], []
    for w in words:
#         print(w)
        context = [0]*block_size
        for ch in w :
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
        
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(results)
n1 = int(0.8*len(results))
n2 = int(0.9*len(results))

Xtr, Ytr = build_dataset(results[:n1])
Xdev, Ydev = build_dataset(results[n1:n2])
Xte, Yte = build_dataset(results[n2:])

torch.Size([5248, 3]) torch.Size([5248])
torch.Size([656, 3]) torch.Size([656])
torch.Size([656, 3]) torch.Size([656])


In [9]:
for x,y in zip(Xtr[:20], Ytr[:20]):
    print([itos[ix.item()] for ix in x], '-->', itos[y.item()])
#   print([itos[ix.item()] for ix in x]), '-->', itos[y.item()])

[0, 0, 0] --> 4
[0, 0, 4] --> 10
[0, 4, 10] --> 17
[4, 10, 17] --> 27
[10, 17, 27] --> 34
[17, 27, 34] --> 43
[27, 34, 43] --> 18
[34, 43, 18] --> 26
[0, 0, 0] --> 3
[0, 0, 3] --> 13
[0, 3, 13] --> 17
[3, 13, 17] --> 32
[13, 17, 32] --> 38
[17, 32, 38] --> 44
[32, 38, 44] --> 43
[38, 44, 43] --> 45
[0, 0, 0] --> 20
[0, 0, 20] --> 21
[0, 20, 21] --> 31
[20, 21, 31] --> 32


In [10]:
torch.manual_seed(myseed+46); # seed rng for reproducibility

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, BatchNormalization, Dropout
embedding_dim=16
max_sequence_length=block_size
h1 = 256
h2 = 128
# Define the model
model = Sequential([
    # Embedding Layer
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    
    # Flatten the embedded sequences (or you can use GlobalAveragePooling1D, GlobalMaxPooling1D, etc.)
    tf.keras.layers.Flatten(),
    
    # Fully Connected Neural Network (Dense Layer)
    Dense(units=h1, activation='relu'),
    
    # Batch Normalization
    BatchNormalization(),
    
    # Dropout for regularization
    Dropout(0.5),  # Adjust the dropout rate as needed
    
    # Another Fully Connected Neural Network (Dense Layer)
    Dense(units=h2, activation='relu'),
    
    Dropout(0.5),  # Adjust the dropout rate as needed
    
    # Output Layer (depending on your problem)
    Dense(units=vocab_size, activation='softmax')  # Adjust the activation function for your problem
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 16)             736       
                                                                 
 flatten (Flatten)           (None, 48)                0         
                                                                 
 dense (Dense)               (None, 256)               12544     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                        

In [12]:
model.fit(Xtr.numpy(),Ytr.numpy(), epochs = 10, batch_size = 32, verbose = 1)

Epoch 1/10
164/164 [==============================] - 1s 3ms/step - loss: 3.7629 - accuracy: 0.0373
Epoch 2/10
164/164 [==============================] - 0s 3ms/step - loss: 3.5497 - accuracy: 0.0570
Epoch 3/10
164/164 [==============================] - 0s 3ms/step - loss: 3.3829 - accuracy: 0.0739
Epoch 4/10
164/164 [==============================] - 0s 3ms/step - loss: 3.3306 - accuracy: 0.0817
Epoch 5/10
164/164 [==============================] - 0s 3ms/step - loss: 3.2726 - accuracy: 0.0777
Epoch 6/10
164/164 [==============================] - 0s 3ms/step - loss: 3.2440 - accuracy: 0.0938
Epoch 7/10
164/164 [==============================] - 1s 3ms/step - loss: 3.2286 - accuracy: 0.1016
Epoch 8/10
164/164 [==============================] - 1s 5ms/step - loss: 3.1977 - accuracy: 0.1027
Epoch 9/10
164/164 [==============================] - 1s 8ms/step - loss: 3.1812 - accuracy: 0.1067
Epoch 10/10
164/164 [==============================] - 3s 16ms/step - loss: 3.1653 - accuracy: 0.100

In [13]:
loss, accuracy = model.evaluate(Xtr.numpy(),Ytr.numpy())

164/164 [==============================] - 4s 15ms/step - loss: 2.9523 - accuracy: 0.1652


In [14]:
loss, accuracy = model.evaluate(Xdev.numpy(), Ydev.numpy())

21/21 [==============================] - 1s 9ms/step - loss: 3.1479 - accuracy: 0.0854


In [15]:
loss, accuracy = model.evaluate(Xte.numpy(), Yte.numpy())

21/21 [==============================] - 0s 11ms/step - loss: 3.2033 - accuracy: 0.0884


In [16]:
import math
-math.log(1/46)

3.828641396489095

In [22]:
# sample from the model
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
import numpy as np

g = torch.Generator().manual_seed(myseed+12)#2147483647 + 12)
start = []

for i in range(20):

    out = []    
    context = [0] * block_size # initialize with all ...
    
    #context = [np.random.randint(1, 46)] * block_size # initialize with all ...
    #out = context
    while True:
        probs = model.predict([context],verbose=0);
        probs = torch.from_numpy(probs)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        if (not ix in out):
            out.append(ix)
        if len(out) == 6: #do until .
            break
    print(out)

[7, 16, 26, 29, 37, 17]
[2, 3, 8, 34, 36, 39]
[2, 11, 14, 19, 22, 30]
[5, 10, 15, 24, 34, 35]
[3, 6, 10, 13, 18, 19]
[11, 12, 21, 45, 7, 37]
[2, 36, 41, 45, 34, 20]
[1, 6, 11, 29, 40, 43]
[4, 14, 18, 22, 43, 42]
[1, 6, 9, 39, 11, 30]
[15, 13, 37, 38, 4, 24]
[17, 28, 34, 36, 4, 25]
[1, 12, 31, 40, 44, 13]
[1, 16, 25, 32, 34, 40]
[2, 5, 16, 31, 38, 23]
[1, 2, 19, 20, 25, 28]
[6, 11, 23, 32, 39, 45]
[2, 26, 31, 36, 35, 44]
[23, 28, 10, 41, 11, 38]
[5, 32, 41, 7, 8, 34]
